In [ ]:
"""Data Aumentation
   1. 翻转变换 flip
   2. 随机修剪 random crop
   3. 色彩抖动 color jittering
   4. 平移变换 shift
   5. 尺度变换 scale
   6. 对比度变换 contrast
   7. 噪声扰动 noise
   8. 旋转变换/反射变换 Rotation/reflection
"""
import matplotlib.pyplot as plt 
from PIL import Image, ImageEnhance, ImageOps, ImageFile
import glob
import numpy as np
import random
import threading, os, time
from multiprocessing import Pool

# import logging
# logger = logging.getLogger(__name__)
# ImageFile.LOAD_TRUNCATED_IMAGES = True

U_NORM_PARAM = 0.435912
V_NORM_PARAM = 0.614777

class ImageProcess:
    """
    Eight ways to augment image data
    """


    def __init__(self):
        pass

    @staticmethod
    def openImage(image):
        return Image.open(image, mode="r")
    
    @staticmethod
    def saveImage(image, path):
        image.save(path)
        
    @staticmethod
    def checkDir(src_path, dst_path):
        if not os.path.isdir(src_path):
            print(src_path, "is not directory!")
            return False
        if not os.path.isdir(dst_path):
            os.mkdir(dst_path)
            print("make new", dst_path, "path!")   
        return True
    
    @staticmethod
    def getAllFiles(rootDir):
        files = []
        fileList = os.listdir(rootDir) # list all directories and files

        for i in fileList:

            path = os.path.join(rootDir, i)
            if os.path.isdir(path):
                files.extend(ImageProcess.getAllFiles(path))
            elif os.path.isfile(path):
                files.append(path)
        return files

    @staticmethod
    def rgb_to_yuv(image):
        """
        convert image in RGB format to YUV
        :param image PIL class image
        :return: numpy array of image after conversion
        """
        rgb_img = np.array(image)
        _r = rgb_img[:, :, 0]
        _g = rgb_img[:, :, 1]
        _b = rgb_img[:, :, 2]

        _y = 0.299 * _r + 0.587 * _g + 0.114 * _b
        _u = 0.492 * (_b - _y)
        _v = 0.877 * (_r - _y)

        _u = _u / (U_NORM_PARAM * 2) + 0.5
        _v = _v / (V_NORM_PARAM * 2) + 0.5

        # given an interval, values outside the interval are clipped to the interval edges.
        _y = _y.clip(0, 1)
        _u = _u.clip(0, 1)
        _v = _v.clip(0, 1)
        
        # reshape to 3-dimension
        _y = _y.reshape((280, 200, 1))
        _u = _u.reshape((280, 200, 1))
        _v = _v.reshape((280, 200, 1))
        
        # concatenate the vector with the 2nd axis
        yuv_img = np.concatenate((_y, _u, _v), axis=2)

        return yuv_img

    def yuv_to_rgb(image):
        """
        convert image in YUV format to RGB
        :param image PIL class image
        :return: numpy array of image after conversion
        """

        yuv_img = np.array(image)

        _y = yuv_img[:, :, 0]
        _u = yuv_img[:, :, 1]
        _v = yuv_img[:, :, 2]

        _u = (_u - 0.5) * U_NORM_PARAM * 2
        _v = (_v - 0.5) * V_NORM_PARAM * 2

        _r = _y + 1.14 * _v
        _g = _y - 0.395 * _u - 0.581 * _v
        _b = _y + 2.033 * _u

        # given an interval, values outside the interval are clipped to the interval edges.
        _r = _r.clip(0, 1)
        _g = _g.clip(0, 1)
        _b = _b.clip(0, 1)
        
        # reshape to 3-dimension
        _r = _r.reshape((280, 200, 1))
        _g = _g.reshape((280, 200, 1))
        _b = _b.reshape((280, 200, 1))
        
        # concatenate the vector with the 2nd axis
        rgb_img = np.concatenate((_r, _g, _b), axis=2)
        return rgb_img
        
    @staticmethod
    def imageResize(image, height=280, width=200, mode=Image.BICUBIC):
        """
        resize the image to specific size
        :param mode 邻近插值,双线性插值,双三次B样条插值(default)
        :param image PIL class image
        :return: image after resize
        """
        return image.resize((width, height), mode)

    @staticmethod
    def randomRotation(image, mode=Image.BICUBIC):
        """
        rotate image at random angle
        :param mode 邻近插值,双线性插值,双三次B样条插值(default)
        :param image PIL class image
        :return: image after rotation
        """
        random_angle = np.random.randint(-20, 20)
        return image.rotate(random_angle, mode)

    @staticmethod
    def randomCrop(image):
        """
        considering the size of pic (200, 280), use windows ranged from 40 to 70 to crop randomly
        :param image: PIL class image
        :return: image after crop

        """
        image_width = image.size[0]
        image_height = image.size[1]
        crop_win_size = np.random.randint(190, 200)
        random_region = (
            (image_width - crop_win_size) >> 1, 
            (image_height - crop_win_size) >> 1, 
            (image_width + crop_win_size) >> 1,
            (image_height + crop_win_size) >> 1
        )
        image = image.crop(random_region)
        return ImageProcess.imageResize(image)


    @staticmethod
    def randomColor(image):
        """
        use color jittering
        :param image: PIL class image
        :return: image of
        """
        # No restriction to random_factor, 1.0 is origin
        
        # adjust the saturation
        random_factor = np.random.uniform(0.7, 1.6)
        minus = np.random.choice([1, -1])
        random_factor *= minus
        color_image = ImageEnhance.Color(image).enhance(random_factor)  
        
        # adjust the brightness
        random_factor = np.random.uniform(0.6, 1.3) 
        brightness_image = ImageEnhance.Brightness(color_image).enhance(random_factor)  
        
        # adjust the contrastion
        random_factor = np.random.uniform(0.4, 1.8)
        contrast_image = ImageEnhance.Contrast(brightness_image).enhance(random_factor)
        
        # adjust the sharpness
        random_factor = np.random.uniform(0, 2)
        sharpness_image = ImageEnhance.Sharpness(contrast_image).enhance(random_factor)
        
        return sharpness_image

    @staticmethod
    def randomGaussian(image, mean=0.0, stddev=0.3):
        """
        use Gaussian nosiy to process image
        :param image:
        :return:
        """

        def gaussianNoisy(im, mean=0.0, stddev=0.3):
            """
            sub function
            :param im: image in 1 dimension
            :param mean: mean
            :param stddev: stddev
            :return:
            """
            for _i in range(len(im)):
                im[_i] += random.gauss(mean, stddev)
            return im

#       # convert PIL image to array (np.asarray will request no memory compared to np.array)
#         img = np.asarray(image)
#       # set writable flag to True
#         img.flags.writeable = True

        img = np.array(image)

        width, height = img.shape[:2]
        img_r = gaussianNoisy(img[:, :, 0].flatten(), mean, stddev)
        img_g = gaussianNoisy(img[:, :, 1].flatten(), mean, stddev)
        img_b = gaussianNoisy(img[:, :, 2].flatten(), mean, stddev)
        img[:, :, 0] = img_r.reshape([width, height])
        img[:, :, 1] = img_g.reshape([width, height])
        img[:, :, 2] = img_b.reshape([width, height])
        return Image.fromarray(np.uint8(img))

    @staticmethod
    def imgProcess(file, src_path, dst_path, times=20):
        """
        image process for each file
        """
        opsList = {"randomRotation", "randomCrop", "randomColor", "randomGaussian"}
        funcMap = {"randomRotation": ImageProcess.randomRotation,
               "randomCrop": ImageProcess.randomCrop,
               "randomColor": ImageProcess.randomColor,
               "randomGaussian": ImageProcess.randomGaussian
               }

        file_path = src_path + file
        if not os.path.isfile(file_path):
            print("image doesn't exist!")
            return
        im = ImageProcess.openImage(file_path)
        # resize the image
        im = ImageProcess.imageResize(im)
        im_name = file.split('.')[0]

        for i in range(times):
            im_cur = im
            for func in funcMap.values():
                im_cur = func(im_cur)
            ImageProcess.saveImage(im_cur, dst_path+im_name+"_"+str(i)+".png")
        
        print(file+" process finish!")
    
    @staticmethod
    def multiProcess(src_path, dst_path, times=20):
        """
        The solution of multi-processs
        """
        if ImageProcess.checkDir(src_path, dst_path) is False:
            return
        
        ## get file without path
        
#         files = os.listdir(src_path) 
#         print("file's count:", len(files))
#         pool = Pool()
#         for file in files:
#             pool.apply_async(ImageProcess.imgProcess, args=(file, src_path, dst_path, times))
            
        ## get file with path
        files_paths = ImageProcess.getAllFiles(src_path)
        pool = Pool()
        for file_path in files_paths:
            file = file_path.split('/')[-1]
            src_path = '/'.join(file_path.split('/')[0:-1]) + '/'
            pool.apply_async(ImageProcess.imgProcess, args=(file, src_path, dst_path, times))

        pool.close()
        pool.join()

    @staticmethod
    def multiThread(src_path, dst_path, times=20):
        """
        The solution of multi-thread
        """
        if ImageProcess.checkDir(src_path, dst_path) is False:
            return

#         root, dirs, files = os.walk(src_path)
        files = os.listdir(src_path)

        print(len(files))
        for file in files:
            t = threading.Thread(target=ImageProcess.imgProcess, args=(file, src_path, dst_path, times))
    #         t.setDaemon(True)
            print(file + " is processing...")
            t.start()
            time.sleep(0.2)
        t.join()

    @staticmethod
    def singleThread(src_path, dst_path, times=20):
        """
        The solution of single-process/thread
        """
        if ImageProcess.checkDir(src_path, dst_path) is False:
            return

#         root, dirs, files = os.walk(src_path)
        files = os.listdir(src_path)

        print(len(files))
        for file in files:
            print(file + " is processing...")
            ImageProcess.imgProcess(file, src_path, dst_path, times)


In [ ]:
print(ImageProcess.randomGaussian.__doc__)

## Main function

In [ ]:
if __name__ == '__main__':
    
    src_path = "./data/poster_positive_withChar/"
    dst_path = "./data/poster_positive_data/"
    
    if ImageProcess.checkDir(src_path, dst_path) is False:
        exit()
    
    ImageProcess.multiProcess(src_path, dst_path, 1)
#     multiThread(r"./data/poster_positive/", r"./data/tmp/")
#     singleThread(r"./data/poster_negative/", r"./data/tmp2/")
    
    print("end")

## Only for image resize 

In [ ]:
if __name__ == '__main__':
    
    src_path = "./data/poster_negative_withChar/"
    dst_path = "./data/poster_negative_data/"
    
    if ImageProcess.checkDir(src_path, dst_path) is False:
        exit()
        
    files_paths = ImageProcess.getAllFiles(src_path)
    
    def onlyResize(file_path, dst_path):
        if not os.path.isfile(file_path):
            print("image doesn't exist!")
            return
        im = ImageProcess.openImage(file_path)
        # resize the image
        im = ImageProcess.imageResize(im)
        file_name = file_path.split('/')[-1].split('.')[0]
        file_name = file_name + "_origin.png"
        ImageProcess.saveImage(im, dst_path+file_name)
        print(file_name, "finish!")
    
    pool = Pool()
    
    for file_path in files_paths:
        pool.apply_async(onlyResize, args=(file_path, dst_path))
        
    pool.close()
    pool.join()
    
    print("end process")
    

## Test for image size

In [ ]:
if __name__ == '__main__':
    
    src_path = "./data/poster_positive_data_bak/"
    files = glob.glob(src_path+"*.png")
    np.random.shuffle(files)
    
    for i in range(100):
        im = ImageProcess.openImage(files[i])
        print(np.asarray(im).shape)
    

## test for imageProcess

In [ ]:
if __name__ == '__main__':
    
    path = './data/poster_positive/'
    test_pic_list = os.listdir(path)
    print(len(test_pic_list))
    test_im = Image.open(path+test_pic_list[0])
    print(np.array(test_im).shape)
    opsList = {"randomRotation", "randomCrop", "randomColor", "randomGaussian"}
    funcMap = {"randomRotation": ImageProcess.randomRotation,
           "randomCrop": ImageProcess.randomCrop,
           "randomColor": ImageProcess.randomColor,
           "randomGaussian": ImageProcess.randomGaussian
           }

    im_name = test_pic[0].split('.')[0]

    im_cur = test_im
    for key, func in funcMap.items():
        im_cur = func(im_cur)
        print(key, np.array(im_cur).shape)

## test for RGB-YUV conversion

In [ ]:
if __name__ == '__main__':
    
    path = './data/poster_positive/'
    test_pic_list = os.listdir(path)
    print(len(test_pic_list))
    test_im = Image.open(path+test_pic_list[0]).convert('RGB')


    test_im = ImageProcess.rgb_to_yuv(test_im)
    test_im = ImageProcess.yuv_to_rgb(test_im)


## test for numpy array concatenation

In [ ]:
if __name__ == '__main__':

    a = np.array([1,2,3,4,5])
    b = np.array([7,8,9,10,11])
    c = np.array([12,13,14,15,16])

    d= np.concatenate((a,b,c), axis=0)
    e = [list(a),list(b), list(c)]
    e = np.array(e)
    e = e.T
    # e = np.append(a,b,axis = 1)
    print(e)

## image show

In [ ]:
if __name__ == '__main__':

    path = './data/poster_positive/'
    pic_list = os.listdir(path)
    print(len(pic_list))
    pic = Image.open(path + pic_list[26])
    plt.imshow(np.asarray(pic))
    plt.show()
    # tmp = ImageProcess.randomGaussian(pic)
    tmp = ImageProcess.randomColor(pic)
    plt.imshow(np.asarray(tmp))
    plt.show()

## My image processing

### load image

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

data_path1 = r"./test_data/test1.jpg"
data_path2 = r"./test_data/test2.png"
data_path3 = r"./test_data/test3.jfif"


img1 = cv2.imread(data_path1, cv2.IMREAD_ANYCOLOR)[:,:,::-1]
img2 = cv2.imread(data_path2, cv2.IMREAD_ANYCOLOR)[:,:,::-1]
img3 = cv2.imread(data_path3, cv2.IMREAD_ANYCOLOR)[:,:,::-1]


fig, (ax1, ax2, ax3) = plt.subplots(1,3, dpi=150, figsize=(15,5))
ax1.imshow(img1)
ax2.imshow(img2)
ax3.imshow(img3)
plt.show()

print(img1[200:205, 200:205,:])

### build gray image

In [ ]:
import numpy as np
h = img1.shape[0]
w = img1.shape[1]

In [ ]:
def convertGray(img):
    grayConvertKernel = np.array([0.299, 0.587, 0.114])
    grayImage = np.array(np.dot(img, grayConvertKernel), np.integer)
    print(grayImage.shape)
    plt.imshow(grayImage,cmap="gray")
    return grayImage

grayImage = convertGray(img1)

### gray histogram

In [ ]:
def calcGrayHist(img):
    rtn = np.zeros((256,), np.integer)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            rtn[img[i][j]] += 1
    return rtn

In [ ]:
y = calcGrayHist(img3)
fig, (ax1, ax2) = plt.subplots(1,2, dpi=150, figsize=(10,4))
ax1.imshow(img3)
ax2.bar(np.arange(len(y)),y)
plt.show()

### Scale up the image

In [ ]:
def scaleImage_auto(img, n):
    scaleMatrix = np.array([[n,0,0],[0,n,0]], np.float32)
    scaleImage = cv2.warpAffine(img, scaleMatrix, (w*n, h*n)) # default: inter_linear
    plt.imshow(scaleImage)
    plt.title("automatic scale without linear interpolation")
    plt.show()
    print(scaleImage.shape)
    return scaleImage
    
def scaleImage_manu(img, n):
    scaleImage = np.ones((h*n, w*n, 3), np.integer)
    for i in range(h):
        for j in range(w):
            scaleImage[i*n-n+1:i*n+1, j*n-n+1:j*n+1, :] = img[i,j,:]
    plt.imshow(scaleImage)
    plt.title("manual scale without interpolation")
    plt.show()
    return scaleImage

def fill_pixel(img, i, j , n):
    def linear_inter(p1, p2, x1, x2, x):
        return (x2 - x) * p1 / (x2 - x1) + (x - x1) * p2 / (x2 - x1)
    x = i // n * n
    y = j // n * n
    pixel1 = img[x, y, :]
    pixel2 = img[x + n, y, :]
    pixel3 = img[x, y + n, :]
    pixel4 = img[x + n, y + n, :]
#     print(pixel1, pixel2, pixel3, pixel4, x, y, i, j, n)
    pixel1_2 = linear_inter(pixel1, pixel2, x, x + n, i)
    pixel3_4 = linear_inter(pixel3, pixel4, x, x + n, i)
    pixel_fill = np.array(linear_inter(pixel1_2, pixel3_4, y, y + n, j), np.integer)
#     print(pixel_fill)
    
    img[i,j,:] = pixel_fill
            
def scaleImage_manu_withInter(img, n):
    scaleImage = np.zeros((h*n, w*n, 3), np.integer)
    for i in range(h):
        for j in range(w):
            scaleImage[i*n, j*n, :] = img[i, j, :]
    H = n * h
    W = n * w
    for i in range(H):
        for j in range(W):
            if i % n == 0 and j % n == 0:
                continue
            elif i >= H - n or j >= W - n:
                scaleImage[i, j, :] = img[i // n, j // n, :]
            else:
                fill_pixel(scaleImage, i, j, n)
    plt.imshow(scaleImage)
    plt.title("automatic scale with linear interpolation")
    plt.show()
    return scaleImage

scaleImage1 = scaleImage_auto(img3, 4)
scaleImage2 = scaleImage_manu(img3, 4)
scaleImage3 = scaleImage_manu_withInter(img3, 4)

In [ ]:
for i in range(h * 4):
    for j in range(w * 4):
        if(np.sum(scaleImage1[i,j,:] - scaleImage3[i,j,:]) > 100):
            print(i, j)

### Fast Fourier transform

In [ ]:
img = cv2.imread(data_path1, cv2.IMREAD_GRAYSCALE)
f = np.fft.fft2(img)
fshift = np.fft.fftshift(f)
# absolute, transform to real
# squash number to 0 to 255
s1 = np.log(np.abs(fshift))
fig, (ax1, ax2, ax3) = plt.subplots(1,3, dpi=150, figsize=(10,4))

ax1.imshow(img,'gray')
ax1.set_title('original')
ax2.imshow(s1,'gray')
ax2.set_title('center')
# inverse fft transformation
f1shift = np.fft.ifftshift(fshift)
img_back = np.fft.ifft2(f1shift)
# plurality， unable to display
img_back = np.abs(img_back)
ax3.imshow(img_back,'gray')
ax3.set_title('img back')

## Cosine transform

In [ ]:
img = cv2.imread(data_path1, cv2.IMREAD_GRAYSCALE)

# set array type to float32
img1 = img.astype('float32')

# cosine transform
img_dct = cv2.dct(img1)
img_dct_log = np.log(abs(img_dct))

# inverse transform
img_recor = cv2.idct(img_dct)
print(img_dct.shape)

# image compression
zip_len=300
recor_temp = img_dct[0:zip_len, 0:zip_len]
recor_temp2 = np.zeros(img.shape)
recor_temp2[0:zip_len, 0:zip_len] = recor_temp
# 压缩图片恢复
img_recor1 = cv2.idct(recor_temp2)

# 显示
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, dpi = 120, figsize=(12, 8))
ax1.imshow(img, cmap='gray')
ax1.set_title('original')

ax2.imshow(img_dct_log, cmap='gray')
ax2.set_title('dct transformed')

ax3.imshow(img_recor, cmap='gray')
ax3.set_title('idct transformed')

ax4.imshow(img_recor1, cmap='gray')
ax4.set_title('idct transformed2')

plt.show()